In [2]:
# Imports of needed libraries
from dash import Dash, html, dcc, callback, Output, Input, dash_table
import dash_bootstrap_components as dbc
import plotly.express as px
import pandas as pd
from dash_bootstrap_templates import load_figure_template

In [3]:
# Df All Athletes
athlete_events = pd.read_csv("../data/final_df.csv")

# Df Swedish Athletes
sweden_athletes = pd.DataFrame(athlete_events[athlete_events["NOC"] == "SWE"])

C:\Users\yonis\AppData\Local\Temp\ipykernel_20152\3412453967.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  athlete_events = pd.read_csv("../data/final_df.csv")


In [37]:
participants_medals= athlete_events.groupby(['Year','Country', 'Continent','Country_latitude', 'Country_longitude','Continent_latitude', 'Continent_longitude'], as_index=False)[['Name', 'Medal']].agg(
    {'Name': 'nunique', 'Medal': 'count'})

participants_medals_start = participants_medals.groupby(['Country','Continent' , 
                                                        'Country_latitude', 'Country_longitude','Continent_latitude', 
                                                        'Continent_longitude'], as_index=False)[['Name', 'Medal']].sum()

participants_medals_sport = athlete_events.groupby(['Year', 'Country', 'Sport', 'Continent','Country_latitude', 'Country_longitude','Continent_latitude', 'Continent_longitude'], as_index=False)[['Name', 'Medal']].agg(
    {'Name': 'nunique', 'Medal': 'count'})


In [43]:
participants_medals_sport = athlete_events.groupby(['Year','Country', 'Sport', 'Continent','Country_latitude', 'Country_longitude','Continent_latitude', 'Continent_longitude'], as_index=False)[['Name', 'Medal']].agg(
    {'Name': 'nunique', 'Medal': 'count'})

participants_medals_sport_total = participants_medals_sport.groupby(['Country','Sport' , 'Continent',
                                                        'Country_latitude', 'Country_longitude','Continent_latitude', 
                                                        'Continent_longitude'], as_index=False)[['Name', 'Medal']].sum()

participants_medals_sport_year = participants_medals_sport.groupby(['Year','Country','Sport' , 'Continent',
                                                        'Country_latitude', 'Country_longitude','Continent_latitude', 
                                                        'Continent_longitude'], as_index=False)[['Name', 'Medal']].sum()

In [66]:
# Loading template for graphs
load_figure_template("quartz")

# This initializes a Dash web application, 
# applies the "QUARTZ" Bootstrap theme for styling, 
# and includes a viewport meta tag to ensure proper rendering on various devices.
app = Dash(__name__, 
           external_stylesheets=[dbc.themes.QUARTZ],
               meta_tags=[{"name": "viewport", "content": "width=device-width, initial-scale=1"}]
)

# Needs to be included for deploying on render
server = app.server

# Layout for App
app.layout = html.Div([
    dbc.Row(
        [
            dbc.Col(html.H2("Olympic Games Achievements 1896-2016", className="text-center text-primary")),
        ],
        className="mb-3 mt-3", # Adding marginal bottom and top
    ),
    dbc.Row([
            dcc.Dropdown(id='year_dropdown', className='text-info',
                        multi=True, 
                        options=[year for year in sorted(athlete_events['Year'].unique())], 
                        placeholder='Select Year',
                        style={'width':'150px', 'margin-left': '10px', 'margin-right': '10px'},
            ),
            dcc.Dropdown(id='sport_dropdown', className='text-info',
                        multi=True, 
                        options=[sport for sport in sorted(athlete_events['Sport'].unique())], 
                        placeholder='Select Sport',
                        style={'width':'150px', 'margin-left': '10px', 'margin-right': '10px'},

            ),
            dcc.Dropdown(id='season_dropdown', className='text-info',
                        multi=True, 
                        options=[season for season in sorted(athlete_events['Season'].unique())], 
                        placeholder='Select Season',
                        style={'width':'150px', 'margin-left': '10px', 'margin-right': '10px'})

    ], justify='center', class_name="mb-2 ml-50 mr-50"), 

    dbc.Row([
        dbc.Col(
            dbc.Card(
                [
                    dbc.CardHeader(html.H3("Workning Title Graph One Left", className="text-body-tertiary", id="header_graph_1_left")),
                    dbc.CardBody([
                        dcc.Dropdown(id='country_dropdown_left', 
                                 className='ml-3 mr-3 mb-1 text-info', 
                                 options=[sport for sport in sorted(athlete_events['Sport'].unique())], 
                                 placeholder='Select Sport'),
                                 dcc.Graph(id="graph_1_left", 
                                    figure={}),
                    ]),
                ],
                className="mb-3",
            ), xs=12, sm=11, md=10, lg=5
        ),
        dbc.Col(  
            dbc.Card(
                [
                    dbc.CardHeader(html.H3("Workning Title Graph One Right", className="text-body-tertiary", id="header_graph_2_right")),
                    dbc.CardBody([
                        dcc.Dropdown(id='country_dropdown_right', 
                                 className='ml-3 mr-3 mb-1 text-info', 
                                 options=[year for year in sorted(athlete_events['Year'].unique())], 
                                 placeholder='Select Year'),]),
                ],
                className="mb-3",
            ), xs=12, sm=11, md=10, lg=5
        ),
    ], justify='evenly'),
    dbc.Row([
        dbc.Button("Reset", id='reset-button', className='float-left', n_clicks=0), # Callback to reset page
        dcc.Link("Contributors", href="https://github.com/DeerBay/OS-Project/graphs/contributors", target="_blank"), # "_blank": Opens the linked document in a new tab or window.
    ],style={"margin-top": "20px", "text-align": "center"})
])

@callback(
    Output("graph_1_left", "figure"),
    Input("year_dropdown", "value"),
    Input("sport_dropdown", "value")
)
def figure_one(year, sport):
    if (year in [None, "", []]) and (sport in [None, "", []]):
        fig = px.scatter_mapbox(participants_medals_start, lat="Country_latitude", 
        lon="Country_longitude", size="Name", color="Medal", 
        height=800, width=900, hover_name="Country",  mapbox_style="open-street-map", 
        center=dict(lat=0, lon=0), zoom=1, 
        title="Size according to count of participants=")
    elif year not in [None, "", []] and sport in [None, "", []]:
        df = participants_medals.query("Year==@year")
        fig = px.scatter_mapbox(df, lat="Country_latitude", 
        lon="Country_longitude", size="Name", color="Medal", 
        height=800, width=900, hover_name="Country",  mapbox_style="open-street-map", 
        center=dict(lat=0, lon=0), zoom=1,
        title="Size according to count of participants=")
    elif year in [None, "", []] and sport not in [None, "", []]:
        df = participants_medals_sport.query("Sport==@sport")
        fig = px.scatter_mapbox(df, lat="Country_latitude", 
        lon="Country_longitude", size="Name", color="Medal", 
        height=800, width=900, hover_name="Country",  mapbox_style="open-street-map",
        center=dict(lat=0, lon=0), zoom=1, 
        title="Size according to count of participants=")
    elif year not in [None, "", []] and sport not in [None, "", []]:
        df = participants_medals_sport_year.query("Year==@year")
        df = df.query("Sport==@sport")
        fig = px.scatter_mapbox(df, lat="Country_latitude", 
        lon="Country_longitude", size="Name", color="Medal", 
        height=800, width=900, hover_name="Country",  mapbox_style="open-street-map",
        center=dict(lat=0, lon=0), zoom=1, 
        title="Size according to count of participants=")

    fig.update_mapboxes(bounds_east=180, bounds_west=-180, bounds_north=90, bounds_south=-90)

    return fig


if __name__ == "__main__":
    app.run(debug=True, jupyter_mode="external")
    

Dash app running on http://127.0.0.1:8050/


In [64]:
fig = px.scatter_mapbox(participants_medals_start, lat="Country_latitude", 
        lon="Country_longitude", size="Name", color="Medal", 
        height=800, width=700, hover_name="Country",  mapbox_style="open-street-map", 
        center=dict(lat=0, lon=0), zoom=1,
        title="Size according to count of participants=")

# set bounds to only see one world map and not be able to scroll through repeated maps
fig.update_mapboxes(bounds_east=180, bounds_west=-180, bounds_north=90, bounds_south=-90)
fig.show()